<a href="https://colab.research.google.com/github/bkvkrll/Machine-Learning-in-Business/blob/main/lesson6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("adult.data", header=None)
data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


Cпрогнозировать, превышает ли доход 50 тысяч долларов в год. Wелевая переменная столбец номер 14

2. сделать feature engineering

In [2]:
data[14] = data[14].map({' <=50K': 0, ' >50K': 1}) 
data[14]

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: 14, Length: 32561, dtype: int64

In [3]:
data.iloc[:, -1].value_counts()

0    24720
1     7841
Name: 14, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

x_data = pd.get_dummies(data.iloc[:,:-1])
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

3. обучить любой классификатор (какой вам нравится)

In [5]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [6]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score, precision_recall_curve

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)
f_1 = f1
roc_1 = roc
rec_1= rec
prc_1= prc

Classification results:
f1: 66.79%
roc: 76.56%
recall: 58.19%
precision: 78.36%


4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [8]:
mod_data = pd.get_dummies(data.copy())
pos_ind = np.where(mod_data.loc[:, 14].values == 1)[0]
np.random.shuffle(pos_ind)
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Positives {pos_sample_len} Unlabeling {len(pos_ind)} ')
pos_sample = pos_ind[:pos_sample_len]

Positives 1961 Unlabeling 7841 


In [9]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [11]:
x_data = mod_data.drop([14, 'class_test'], axis = 1) # just the X 
y_labeled = mod_data.loc[:,'class_test'].values # new class (just the P & U)
y_positive = mod_data.loc[:,14].values # original class

5. применить random negative sampling для построения классификатора в новых условиях

In [12]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [13]:
model_rns = xgb.XGBClassifier()

model_rns.fit(sample_train.drop([14, 'class_test'], axis=1).values,
          sample_train.loc[:, 14].values)
y_predict_rns = model_rns.predict(sample_test.drop([14, 'class_test'], axis=1).values)

6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [14]:
pd.DataFrame([evaluate_results(y_test, y_predict), evaluate_results(sample_test.loc[:, 14].values, y_predict_rns)
             ], columns={'f1', 'precision', 'recall', 'roc'}, index={'xgboost', 'rns'}).round(3)

Classification results:
f1: 66.79%
roc: 76.56%
recall: 58.19%
precision: 78.36%
Classification results:
f1: 60.35%
roc: 82.37%
recall: 90.91%
precision: 45.16%


,f1,roc,precision,recall
rns,0.668,0.766,0.582,0.784
xgboost,0.603,0.824,0.909,0.452
